In [3]:
# !pip install -U git+https://github.com/dgunning/cord19.git

In [26]:
# !pip install git+https://github.com/neuml/cord19q

In [27]:
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

# Following the pipeline of 
### https://www.kaggle.com/dgunning/building-a-cord19-research-engine-with-bm25

In [2]:
%load_ext autotime
import os

os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask
os.environ['MODIN_MEMORY'] ='2000000000000'
import modin.pandas as pd

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.75 s


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

time: 10.1 s


In [4]:
from cord import ResearchPapers

research_papers = ResearchPapers.load(data_dir='/home/syncthing/others/covid/data/')

Loading metadata from /home/syncthing/others/covid/data/CORD-19-research-challenge
Cleaning metadata
Applying tags to metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead
Finished Indexing in 53.0 seconds
time: 1min 11s


In [5]:
research_papers

time: 158 ms


In [28]:
research_papers.metadata.head()

,cord_uid,sha,source,title,doi,pmcid,pubmed_id,license,abstract,published,...,has_text,full_text_file,url,when,covid_related,virus,coronavirus,sars,index_tokens,antivirals
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,...,True,biorxiv_medrxiv,https://doi.org/10.1101/001727,6 years ago,False,False,False,False,"[next-generation, sequencing, increasingly, us...",
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,...,True,biorxiv_medrxiv,https://doi.org/10.1101/003889,6 years ago,False,False,False,True,"[emerging, disease, infectious, epidemic, caus...",
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,...,True,biorxiv_medrxiv,https://doi.org/10.1101/006866,6 years ago,False,False,False,False,"[germline, variation, immunoglobulin, gene, ig...",
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,...,True,biorxiv_medrxiv,https://doi.org/10.1101/007476,6 years ago,False,True,False,False,"[deep, sequencing, clinical, samples, establis...",
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,...,True,biorxiv_medrxiv,https://doi.org/10.1101/010389,5 years ago,False,True,False,False,"[developing, methods, reconstruct, transmissio...",


In [19]:
import pandas as pd

research_papers.metadata.to_pickle('covid19_kaggle.pkl')
research_papers.metadata.to_csv('covid19_kaggle.csv')

In [6]:
research_papers.metadata.columns

Index(['cord_uid', 'sha', 'source', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'published', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'has_text',
       'full_text_file', 'url', 'when', 'covid_related', 'virus',
       'coronavirus', 'sars', 'index_tokens', 'antivirals'],
      dtype='object')

time: 8.44 ms


In [7]:
research_papers.metadata['journal'].nunique()

4886

time: 15.7 ms


In [8]:
research_papers.metadata['journal'].unique()

array(['', 'Chin Med J (Engl)', 'J Clin Med', ...,
       'Medical Image Analysis', 'Pediatric Reports. 2020;12(1)',
       'Current Opinion in Chemical Engineering'], dtype=object)

time: 15.4 ms


# search functionality

In [9]:
research_papers.searchbar('antiviral treatment')

Output()

time: 1.42 s


In [10]:
research_papers.match('H[0-9]N[0-9]')

time: 153 ms


# For task 4

## item 1: Effectiveness of drugs being developed and tried to treat COVID-19 patients.
### Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocyclinethat that may exert effects on viral replication.

In [35]:
inhibitor = research_papers.contains('inhibitors', column='abstract').since_sarscov2()
inhibitor

time: 523 ms


In [33]:
naproxen = research_papers.contains('naproxen',column='abstract')
naproxen

time: 305 ms


In [32]:
clarithromycin = research_papers.contains('clarithromycin', column='abstract')
clarithromycin

time: 281 ms


In [30]:
minocyclinethat = research_papers.search('minocyclinethat')
minocyclinethat

time: 221 ms
